In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from dataclasses import dataclass
from math import radians
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import trange
import ipywidgets as widgets

from cw.context import time_it
from cw.simulation import Simulation, StatesBase, AB3Integrator, BatchLogger, Logging
from cw.filters import smooth_signal

from topone.sim_post_processing import sim_post_processing
from topone.dynamics_1 import Dynamics1, Stage

from environment import Environment
from actor_critic_agent import ActorCriticAgent

In [3]:
@dataclass
class States(StatesBase):
    t: float = 0
    command_engine_on: bool = False
    command_drop_stage: bool = False

    command_theta_e: float = 0.0
    command_gamma_e_dot: float = 0.0
    command_gamma_e: float = 0.0

    gii: np.ndarray = np.zeros(2)
    xii: np.ndarray = np.zeros(2)
    vii: np.ndarray = np.zeros(2)
    aii: np.ndarray = np.zeros(2)
    tci: np.ndarray = np.eye(2)
    vic: np.ndarray = np.zeros(2)
    fii_thrust: np.ndarray = np.zeros(2)
    theta: float = 0.
    theta_dot: float = 0.
    theta_e: float = 0.
    mass: float = 0.
    mass_dot: float = 0.
    h: float = 0.
    engine_on: bool = False
    stage_state: int = 0
    stage_idx: int = 0
    stage_ignitions_left: int = 0
    gamma_i: float = 0.
    gamma_e: float = 0.
    latitude: float = 0.

    reward: float = 0.
    score: float = 0.
    done: bool = False

    delta_v: float = 0.

    def get_y_dot(self):
        y = np.empty(7)
        y[:2] = self.vii
        y[2:4] = self.aii
        y[4] = self.theta_dot
        y[5] = self.mass_dot
        y[6] = self.reward
        return y

    def get_y(self):
        y = np.empty(7)
        y[:2] = self.xii
        y[2:4] = self.vii
        y[4] = self.theta
        y[5] = self.mass
        y[6] = self.score
        return y

    def set_t_y(self, t, y):
        self.t = t
        self.xii = y[:2]
        self.vii = y[2:4]
        self.theta = y[4]
        self.mass = y[5]
        self.score = y[6]
        

In [4]:
environment = Environment(target_time_step=0.01)
agent = ActorCriticAgent(
    path="./set_1",
    alpha=0.01,
    gamma=0.99,
    environment=environment,
)

simulation = Simulation(
    states_class=States,
    integrator=AB3Integrator(
        h=0.01,
        rk4=False,
        fd_max_order=1),
    modules=[
        Dynamics1(
            surface_diameter=1737.4e3,
            mu=4.9048695e12,
            stages=(
                Stage(
                    dry_mass=1,
                    propellant_mass=0.02,
                    specific_impulse=100,
                    thrust=2*1.7,
                    n_ignitions=None
                ),
            ),
            initial_altitude=1.,
            initial_theta_e=radians(90),
            initial_latitude=radians(0),
        ),
        environment
    ],
    logging=Logging(),
    initial_state_values=None,
)

In [5]:
environment.start_simulation_thread(1000)

In [6]:
agent.train(1000, 60000, 5)

In [10]:
training_history = pd.DataFrame.from_records(agent.training_history, columns=agent.training_history[0]._fields)

In [11]:
training_history

,reward_sum,save_idx,process_time
0,103.300659,0,2.347480
1,4604.539062,1,1.224804
2,106.152466,1,0.125889
3,96.952240,1,0.124668
4,97.194588,1,0.105119
...,...,...,...
4270,30011.396484,500,1.431007
4271,30011.396484,500,1.261516
4272,29895.496094,500,1.371902
4273,30011.396484,500,1.443712


In [12]:
fig, ax1 = plt.subplots()
training_history.reward_sum.plot()
plt.plot(smooth_signal(training_history.reward_sum, wn=0.1))

ax2 = ax1.twinx()
training_history.save_idx.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:label='bfa94151-befa-4a7d-ae36-935d3d1ae088'>

In [9]:
agent.load(500)

In [12]:
fig, ax1 = plt.subplots()
training_history.reward_sum.plot()
plt.plot(smooth_signal(training_history.reward_sum, wn=0.005))

ax2 = ax1.twinx()
training_history.save_idx.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:label='9ed63f12-c6f0-440f-acba-0dc5511630f4'>

In [9]:
agent.save()

In [6]:
# agent.display_greedy_policy()

In [10]:
environment.last_results

<IPython.core.display.JSON object>

In [20]:
environment.step(0)

(array([ True, False, False]), 0, False, {'time': 0.02})

In [6]:
# environment.gamma_controller.k_p = 4.
# environment.gamma_controller.k_i = .1
# environment.gamma_controller.k_d = 0.1
# environment.theta_controller.k_p = 10.
# environment.theta_controller.k_i = 0.
# environment.theta_controller.k_d = 0.
r = agent.run_episode_greedy(1000)
# r

In [7]:
plt.figure()
r.gamma_e.plot.line(label="gamma_i")
r.command_gamma_e.plot.line(label="command_gamma_e")
plt.legend()

plt.figure()
r.theta_e.plot.line(label="theta_e")
r.command_theta_e.plot.line(label="command_theta_e")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure()
r.theta_dot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …